# Analise exploratória

In [1]:
from pathlib import Path
import sys

parent = Path().absolute().parents[0].as_posix()

sys.path.insert(0, parent)

from tqdm import tqdm

import pandas as pd
import numpy as np

import yake
import spacy
import pickle

from nlpiper.core import Compose
from nlpiper.transformers import cleaners
from nlpiper.core import Document


from gensim.corpora.dictionary import Dictionary
from gensim import models 

from sklearn.cluster import DBSCAN, KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from resources.stopwords import WORDS

2022-04-28 10:19:28.637726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-28 10:19:28.637750: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
#data_raw = pd.read_csv('../data/scraping_data_v2.csv.gz', compression='gzip')
data_cleaned = pd.read_csv('../data/processed/docs_cleaned_w_categories.csv.gz', compression='gzip')
publico_articles = pd.read_csv('../data/processed/publico_docs_cleaned_w_keywords.csv.gz', compression='gzip')
data_political_parties = pd.read_csv('../data/scraping_political_parties.csv.gz', compression='gzip')

In [3]:
data_raw[data_raw.year >= 2014].source.value_counts()

observador.pt    147591
expresso.pt       13037
publico.pt         6054
jn.pt              5678
Name: source, dtype: int64

In [6]:
data_raw[(data_raw.source == 'expresso.pt') & (data_raw.year == 2014)]

,city,title,content,year,tstamp,link,source
1404,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141004011519,https://arquivo.pt/wayback/20141004011519/http...,expresso.pt
1405,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141003170733,https://arquivo.pt/wayback/20141003170733/http...,expresso.pt
1406,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141016071819,https://arquivo.pt/wayback/20141016071819/http...,expresso.pt
1407,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141016071101,https://arquivo.pt/wayback/20141016071101/http...,expresso.pt
1408,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141016064106,https://arquivo.pt/wayback/20141016064106/http...,expresso.pt
1409,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141021235743,https://arquivo.pt/wayback/20141021235743/http...,expresso.pt
1410,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141004005435,https://arquivo.pt/wayback/20141004005435/http...,expresso.pt
1411,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141003174041,https://arquivo.pt/wayback/20141003174041/http...,expresso.pt
1412,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141003143310,https://arquivo.pt/wayback/20141003143310/http...,expresso.pt
1413,Lisboa,301 Moved Permanently,301 Moved Permanently Moved Permanently The do...,2014,20141003151457,https://arquivo.pt/wayback/20141003151457/http...,expresso.pt


In [4]:
data_cleaned.source.value_counts()

observador.pt    47262
expresso.pt       7136
publico.pt         522
jn.pt               70
Name: source, dtype: int64

In [ ]:
data_political_parties.head()

In [ ]:
data_political_parties[(data_political_parties.city == 'Lisboa') & (data_political_parties.year==2015)]

In [ ]:
data_political_parties.year.value_counts()

In [8]:
data_cleaned.head()

,city,title,content,year,tstamp,link,source,link_len,check,content_p,title_p,category
0,Lisboa,"Oceanário de Lisboa vai ter novo edifício ""tra...","Oceanário de Lisboa vai ter novo edifício ""tra...",2014,20140516081533,https://arquivo.pt/wayback/20140516081533/http...,publico.pt,157,True,oceanario lisboa novo edificio tranquilo anon...,oceanario lisboa novo edificio tranquilo anon...,Economia
1,Lisboa,Câmara de Loures paga mais do dobro do que Lis...,Câmara de Loures paga mais do dobro do que Lis...,2014,20141023080605,https://arquivo.pt/wayback/20141023080605/http...,publico.pt,155,True,camara loures paga dobro lisboa refeicoes esco...,camara loures paga dobro lisboa refeicoes esco...,Politica
2,Lisboa,Deputado Bacelar Gouveia candidata-se à presid...,Deputado Bacelar Gouveia candidata-se à presid...,2014,20141021142620,https://arquivo.pt/wayback/20141021142620/http...,publico.pt,165,True,deputado bacelar gouveia candidatase presidenc...,deputado bacelar gouveia candidatase presidenc...,Politica
3,Lisboa,“Darth Vaders” do blogue 31 da Armada hasteara...,“Darth Vaders” do blogue 31 da Armada hasteara...,2014,20141021111635,https://arquivo.pt/wayback/20141021111635/http...,publico.pt,172,True,darth vaders blogue armada hastearam bandeira...,darth vaders blogue armada hastearam bandeira...,Politica
4,Lisboa,Ordem quer que CML esclareça ranking de arquit...,Ordem quer que CML esclareça ranking de arquit...,2014,20141021180751,https://arquivo.pt/wayback/20141021180751/http...,publico.pt,162,True,ordem quer cml esclareca ranking arquitectos a...,ordem quer cml esclareca ranking arquitectos a...,Justica


In [4]:
publico_articles.head(1)

,city,main_tag,tag,title,sub_title,body,year,category,body_p,keywords
0,Lisboa,ciencia,HISTÓRIA,A quinta avenida do século XVI ficava em Lisboa,Dois quadros descobertos em 2009 originaram um...,"No século XVI, a Rua Nova dos Mercadores era u...",2015.0,Ciencia,seculo xvi rua nova mercadores pequena babel. ...,"['Rua', 'Lisboa', 'Annemarie Jordan', 'Jordan ..."


In [16]:
def string_finder(text, target_word):
    begin = text.find(target_word)
    end = begin + len(target_word)
    return begin, end, target_word


In [24]:
import ast
for keyword in ast.literal_eval(publico_articles.iloc[0].keywords):
    print(keyword)
    print(string_finder(publico_articles.iloc[0].body, keyword))

Rua
(17, 20, 'Rua')
Lisboa
(177, 183, 'Lisboa')
Annemarie Jordan
(550, 566, 'Annemarie Jordan')
Jordan Gschwend
(560, 575, 'Jordan Gschwend')
Mercadores
(30, 40, 'Mercadores')


In [33]:
keywords = ast.literal_eval(publico_articles.iloc[0].keywords)
keywords = [k.split(' ') for k in keywords]
keywords = [item for sublist in keywords for item in sublist]
keywords


['Rua', 'Lisboa', 'Annemarie', 'Jordan', 'Jordan', 'Gschwend', 'Mercadores']

In [51]:
def build_keyword_annotations(doc, keywords):
    article = []
    for word in doc.split(' '):
        article.append(word)
        if word in keywords:
            article.append((word, "#8ef"),)
    return article

In [ ]:
article

In [37]:
from annotated_text import annotated_text


In [53]:
annotated_text(*article)

In [14]:
publico_articles['source'] = ['publico.pt'] * len(publico_articles)
all_sources = pd.concat([data_cleaned[['city', 'year', 'title', 'category', 'source']], publico_articles[['city', 'year', 'title', 'category', 'source']]])
all_sources.shape

(62604, 5)

In [15]:
all_sources.head()

,city,year,title,category,source
0,Lisboa,2014.0,"Oceanário de Lisboa vai ter novo edifício ""tra...",Economia,publico.pt
1,Lisboa,2014.0,Câmara de Loures paga mais do dobro do que Lis...,Politica,publico.pt
2,Lisboa,2014.0,Deputado Bacelar Gouveia candidata-se à presid...,Politica,publico.pt
3,Lisboa,2014.0,“Darth Vaders” do blogue 31 da Armada hasteara...,Politica,publico.pt
4,Lisboa,2014.0,Ordem quer que CML esclareça ranking de arquit...,Justica,publico.pt


In [16]:
articles_general = all_sources.groupby(['city','year']).size().reset_index(name='N')
subset_categories = all_sources.groupby(['city', 'category' ,'year']).size().reset_index(name='N')
subset_articles_general = all_sources.groupby(['city', 'source','year']).size().reset_index(name='N')
subset_articles_general_by_category = all_sources.groupby(['city', 'source', 'category','year']).size().reset_index(name='N')

articles_general.to_csv('../data/dashboard/articles_general.csv', index=False)
subset_categories.to_csv('../data/dashboard/subset_categories.csv', index=False)
subset_articles_general.to_csv('../data/dashboard/subset_articles_general.csv', index=False)
subset_articles_general_by_category.to_csv('../data/dashboard/subset_articles_categories.csv', index=False)


In [ ]:
subset_articles[subset_articles.year==2019]

### Filter Links

In [ ]:
data[(data.source == 'publico') & (data.year == 2019)].link.iloc[-10:].to_list()

In [ ]:
data['link_len'] = data['link'].apply(lambda val: len(val))

In [ ]:
data['link_len'].mean()

## Articles Headers and Footers 

Publico:
- Continuar a ler Assinar o Público é participar na construção de um país melhor O PÚBLICO nunca foi tão lido. Todos os meses passam pelo nosso online mais de 6.5 milhões de visitantes. Para nós, este número confirma a importância do nosso trabalho. Queremos produzir mais e melhor informação, com a liberdade de sempre e sem abdicar da diversidade de opiniões que enriquece uma sociedade livre. Queremos reforçar a nossa investigação para garantir um escrutínio mais eficaz dos poderes. Precisamos que se junte a nós neste esforço. A verdade, o pluralismo, a justiça, a solidariedade ou a abertura ao mundo são valores que partilhamos consigo. Sinta-se ainda mais parte deste projecto cívico. Pense bem, pense Público. Assine já Tópicos Local Transportes Lisboa transporte rodoviário Área Metropolitana de Lisboa Loures Odivelas Vila Franca de Xira Alcochete Almada Amadora Barreiro Cascais Mafra Moita Montijo Oeiras Palmela Seixal Sesimbra Setúbal Sintra Partilhar notícia 1 partilhas Partilhar no Facebook Partilhar no Twitter Partilhar no WhatsApp Partilhar no Messenger Partilhar no Google+ Partilhar no LinkedIn Partilhar no Pinterest Enviar por email Imprimir Guardar Comentar Sugerir correcção × Comentários Aprovados 0 Pendentes 0 Não há comentários Seja o primeiro a comentar. Mais comentários Não há comentários pendentes Em destaque Mais artigos A carregar... Público Siga-nos Newsletters Facebook Twitter Instagram LinkedIn YouTube RSS Actualidade Política Sociedade Local Economia Mundo Cultura Desporto Ciência Tecnologia Opinião PGlobal Multimédia Podcasts Secções P2 Ípsilon Culto Fugas P3 Cidades Inimigo Público Lazer Cinecartaz Guia do Lazer Programação de TV Quiosque Aplicações Loja Iniciativas Novos Projectos Serviços Imobiliário Sobre Ficha Técnica Estatuto Editorial Autores Contactos Provedor do Leitor Público+ Publicidade Assinaturas Assinar Conteúdos exclusivos Descontos para assinantes Edição impressa Cartão Público Email marketing por @ 2019 PÚBLICO Comunicação Social SA Ajuda Termos e Condições Política de Privacidade Principais Fluxos Financeiros Estrutura Accionista A Mensagem Nónio Ir para o conteúdo Ir para navegação principal × Pesquise no Público × Conta Entrar Pesquisar Edição impressa Assinar A minha conta Biblioteca Perfil público Moderação Cartão Público Edição Impressa Conteúdos exclusivos Sair Em destaque Ambiente Saúde Demografia EUA Incêndios Água Actualidade Política Sociedade Local Economia Mundo Cultura Desporto Ciência Tecnologia Opinião PGlobal Multimédia Podcasts Cidades Secções P2 Ípsilon Culto Fugas P3 Cidades Bartoon Inimigo Público Lazer Cinecartaz Guia do Lazer Programação de TV Siga-nos Newsletters Facebook Twitter Instagram LinkedIn YouTube RSS Assinaturas Assinar Conteúdos exclusivos Edição impressa Descontos para assinantes Cartão Público Serviços Emprego Imobiliário Quiosque Aplicações Loja Iniciativas Novos Projectos Sobre Ficha Técnica Estatuto Editorial Autores Contactos Provedor do Leitor Público+ Publicidade Entrar Email Palavra-chave A sua conta não se encontra ativa. Clique aqui para receber um e-mail de ativação de conta. Esqueceu-se da sua palavra-chave? Lembrar-se dos meus dados? Não tem uma conta? Registe-se gratuitamente × × ×
- Assine já Público © 2015 Público Comunicação Social SA Facebook Twitter Google+ RSS Email marketing por Mapa do site Secções Portugal Economia Mundo Cultura-Ípsilon Desporto Ciência Tecnologia Opinião Multimédia Edição Impressa Tópicos Sites Público 2 Fugas Life&Style P3 Ípsilon Cinecartaz Guia do Lazer Inimigo Público Serviços Meteorologia Loja Emprego Jogos TV Classificados Imobiliário Iniciativas Novos projectos QUIOSQUE PÚBLICO Assinaturas Aplicações Mobile Sites Mobile Tablet Kindle Informações Novo site Contactos Ficha Técnica Autores Ajuda Comentários e Inquéritos Público+ Provedor do Leitor Termos e Condições Política de Privacidade Publicidade 30 dias apenas 1€ * Assine o Público Online e continue a ler e a pensar sem limites. Ao atingir o limite de artigos do Público Online mostrou como é importante para si a excelência e qualidade do jornalismo em Portugal. * Para pagamentos por Visa/PayPal/débito directo. Restantes meses €9,99/cada. Assine já e leia o Público durante 30 dias por apenas 1€. Já é assinante? Inicie a sessão Perguntas Frequentes Nós Ligamos-lhe Apoio Online 8 semanas apenas 0,99€ * Para novos assinantes Assine o Público Online e continue a ler e a pensar sem limites. Ao atingir o limite de artigos do Público Online mostrou como é importante para si a excelência e qualidade do jornalismo em Portugal. * Para pagamentos por Visa/PayPal/débito directo. Restantes meses €9,99/cada. Sem período de fidelização. Assine já e leia o Público durante 8 semanas por apenas 0,99€. Já é assinante? Inicie a sessão Perguntas Frequentes Nós Ligamos-lhe Apoio Online Aprecia o jornalismo do PÚBLICO? Então assine o jornal e tenha o caminho livre para todos os conteúdos online, em qualquer plataforma (mobile, tablet ou web). Não perca as notícias, reportagens, fotografias e trabalhos multimédia que fazem do PÚBLICO um jornal premiado e líder online. Experimente por 1€ * no primeiro mês. Assine já Já é assinante? Inicie sessão * Campanha para pagamentos por VISA / PayPal / débito directo, com renovação mensal de 9,99€. Tem dúvidas? Perguntas Frequentes Esclarecemos as suas questões. Veja os videos explicativos. Nós ligamos-lhe Deixe-nos o seu contato e ligamos-lhe de seguida. Apoio Online Submeta a sua questão e respondemos de imediato por chat.
- Para continuar, inicie sessão ou registe-te já Iniciar Sessão Entrar com o Facebook Entrar com o Twitter Ou Palavra-chave Lembrar-se de mim Esqueceu-se da sua palavra-chave? Registe-se já Registe-se no Público para guardar quantos artigos quiser para ler mais tarde, participar dos inquéritos, tornar-se um moderador de comentários e muito mais. Registar Conteúdo exclusivo Para continuar a ler, inicie sessão ou assine já Palavra-chave Lembrar-se de mim Esqueceu-se da sua palavra-chave? ASSINE O PÚBLICO ONLINE E ENTRE NO MUNDO DAS NOTICIAS SEM LIMITES Nasceu um mundo sem limites: Economia sem limites, política sem limites, cultura sem limites, desporto sem limites, opinião sem limites, vídeo sem limites... Entre neste mundo por apenas 1€*. Assine já * Campanha para pagamento por VISA / PayPal / débito directo. Restantes meses: 9,99€/cada. Público © 2014 Público Comunicação Social SA Facebook Twitter Google+ RSS Mapa do site Secções Portugal Economia Mundo Cultura Desporto Ciência Tecnologia Opinião Multimédia Edição Impressa Tópicos Sites Público 2 Fugas Life&Style P3 Ípsilon Cinecartaz Guia do Lazer Inimigo Público Serviços Meteorologia Loja Emprego Jogos TV Classificados Imobiliário Iniciativas QUIOSQUE PÚBLICO Assinaturas Aplicações Mobile Sites Mobile Tablet Kindle Informações Novo site Contactos Ficha Técnica Autores Ajuda Comentários e Inquéritos Público+ Provedor do Leitor Termos e Condições Política de Privacidade Publicidade Conteúdo exclusivo para assinantes Para continuar a ler, faça login ou assine já. Iniciar Sessão Entrar com o Facebook Entrar com o Twitter Ou Palavra-chave Lembrar-se de mim Esqueceu-se da sua palavra-chave? Assine o Público Ligue 760 10 50 20 Assinatura Diária* Para usufruir de uma assinatura diária e ter acesso a todos os conteúdos exclusivos durante o dia de hoje. Tome nota do código e introduza-o no campo ao lado para iniciar a sessão. Condições [+] Subscrever Outras modalidades A partir de 2,30€ * Número válido apenas para Portugal. Custo da chamada €0,60 + IVA. Esta assinatura é válida até ao final do dia corrente. Assine já × Entre no Mundo das notícias sem limites Experimente por 1€* no primeiro mês e garanta o acesso simples e imediato a todos os conteúdos. * Campanha para pagamento por VISA / PayPal / débito directo. Restantes meses: 9,99€/cada. Leia o Público sem limites Perguntas Frequentes Nós Ligamos-lhe Apoio Online

Expresso:
- Mais Artigos a carregar... Pesquisar Login Perfil Logout Para ler o Semanário e o Diário USAR CÓDIGO Disponível na capa da Revista E ASSINAR Se ainda não tem acesso Edições Diário Semanário Expresso Curto Podcasts Tribuna Site Expresso Início Últimas Colecionáveis Multimédia Política Sociedade Internacional Economia Desporto Cultura Opinião Cartoons Está dito Iniciativas Carro do Ano Global Investment Challenge Entrepreneur Of The Year Classificados Emprego Imobiliário Siga-nos Facebook Twitter Google+ LinkedIn RSS Estatuto editorial Código de Conduta Ficha Técnica do Expresso Política de cookies Termos de utilização Política de privacidade Regras da Comunidade Publicidade Contactos Lei da Transparência Assinar Cartas ao Director Loja © Expresso Impresa Publishing S.A.

- Loja Siga-nos Facebook Twitter RSS Login Logout Perfil Menu Expresso Exclusivos
- Facebook Twitter Email Whatsapp Mais Google+ Linkedin Pinterest Link:

- Partilhar no Facebook Partilhar no Twitter Partilhar no Google+
- PÚBLICO Abrir menu Secções Abrir pesquisa Pesquisa Edição impressa Público Em destaque P2 Ípsilon Culto Fugas P3 Cinecartaz
- Partilhar notícia \d+ partilhas Partilhar no Facebook Partilhar no Twitter Partilhar no WhatsApp Partilhar no Messenger Partilhar no Google+ Partilhar no LinkedIn Partilhar no Pinterest Enviar por email Guardar Comentar Entrar A minha conta Biblioteca Perfil público Moderação Cartão Público Edição Impressa Conteúdos exclusivos Sair Assine já Subsecções

### Headers

In [ ]:
import re
patterns = [
    r"Partilhar notícia \d+ partilhas Partilhar no Facebook Partilhar no Twitter Partilhar no WhatsApp Partilhar no Messenger Partilhar no Google\+ Partilhar no LinkedIn Partilhar no Pinterest Enviar por email Guardar Comentar Entrar A minha conta Biblioteca Perfil público Moderação Cartão Público Edição Impressa Conteúdos exclusivos Sair Assine já Subsecções",
    r"Partilhar notícia \d+ partilhas Partilhar no Facebook Partilhar no Twitter Partilhar no WhatsApp Partilhar no Messenger Partilhar no Google\+ Partilhar no LinkedIn Partilhar no Pinterest Enviar por email Imprimir Guardar \d+ Comentários"

]
#article = re.sub(pattern, '', df[(df['source'] == 'publico') & (df['year'] == 2019)].content.iloc[5])
#article
# 
article = df[(df['source'] == 'publico') & (df['year'] == 2019)].content.iloc[5]
for pattern in patterns:
    article =  re.sub(pattern, '', article)

article


In [ ]:
df = data[data['link_len'] >=  data['link_len'].mean()]
df[(df['source'] == 'publico') & (df['year'] == 2019)].content.iloc[3]

### Footers

In [ ]:
df[(df['source'] == 'expresso') & (df['year'] == 2019)].content.iloc[3]

In [ ]:
pattern = r"(?=Mais Artigos a carregar...).*"
article = re.sub(pattern, '', df[(df['source'] == 'expresso') & (df['year'] == 2019)].content.iloc[3])
article

## Data Pre-processing

In [ ]:
#docs_on_tokens = np.load('../data/processed/docs_cleaned.npz', allow_pickle=True)['files']
import pickle
with open('../data/processed/docs_cleaned.pickle', 'rb') as handle:
    docs_on_tokens = pickle.load(handle)

In [ ]:
len(docs_on_tokens)

In [ ]:
top_n = 50
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs_on_tokens[0:1000])
feature_array = vectorizer.get_feature_names()

tf_idf_scores = sorted(list(zip(vectorizer.get_feature_names(), 
                                             X.sum(0).getA1())), 
                                 key=lambda x: x[1], reverse=True)

print('tf_idf scores: \n', tf_idf_scores[:top_n])


print('idf values: \n', sorted(list(zip(feature_array,vectorizer.idf_,)),
       key = lambda x: x[1], reverse=True)[:top_n])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs_on_tokens[0:top_n])
feature_array = vectorizer.get_feature_names()
print('Frequency: \n', sorted(list(zip(vectorizer.get_feature_names(), 
                                         X.sum(0).getA1())),
                            key=lambda x: x[1], reverse=True)[:top_n])       


In [ ]:
tf_idf_scores[0]

In [ ]:
len(X.indices)

In [ ]:
len(X.data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["This is very strange",
          "This is very nice"]
vectorizer = TfidfVectorizer(norm='l2')
corpus = vectorizer.fit_transform(corpus)

print(corpus)

In [ ]:
vectorizer.vocabulary_

In [ ]:
print(dict(zip(vectorizer.vocabulary_, vectorizer.idf_)))


## Keyword Detection

In [ ]:
data.link.iloc[-1]

In [ ]:
language = "pt"
max_ngram_size = 3
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 5

custom_kw_extractor = yake.KeywordExtractor(
    lan=language, 
    n=max_ngram_size, 
    dedupLim=deduplication_thresold, 
    dedupFunc=deduplication_algo, 
    windowsSize=windowSize, 
    top=numOfKeywords, 
    features=None
)
keywords = custom_kw_extractor.extract_keywords(docs[0])

for kw in keywords:
    print(kw)
    

In [ ]:
keywords_on_docs = []
for idx in range(len(data)):
    keywords_on_docs.append(custom_kw_extractor.extract_keywords(simple_pipeline(Document(data.content.iloc[idx].lower())).cleaned))

In [ ]:
custom_kw_extractor.extract_keywords(simple_pipeline(Document(data.content.iloc[0].lower())).cleaned)

In [ ]:
with open('../data/processed/docs_keywords.pickle', 'rb') as handle:
        docs_keywords = pickle.load(handle)

In [ ]:
t = []
for keywords in docs_keywords[-50]:
        t.append((keywords[0], model.predict(keywords[0])[0][0]))
t

In [ ]:
from collections import Counter
Counter([v[1] for v in t])

In [ ]:
model.predict(docs_keywords[50][-2][0])[0][0]

## Embeddings

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(sentences=docs_on_tokens, vector_size=100, window=5, min_count=1, workers=4, sg=0, epochs=30)

In [ ]:
model.wv.most_similar('rei', topn=10)

In [ ]:
model.wv.similarity('homen', 'rei')

## Clustering

In [ ]:
from sklearn.cluster import DBSCAN, KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt


In [ ]:
with open('../data/processed/words_embedded.pickle', 'rb') as handle:
    words_emb = pickle.load(handle)

In [ ]:
len(words_emb)

In [ ]:
words_emb[0]

In [ ]:
len(words_emb[0])

In [ ]:
X = [val[0] for val in words_emb]
ks = range(2, 12)
results = {}
for k in ks: 
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    results[k] = silhouette_score(X, kmeans.labels_)
    print(results)

In [ ]:
kmeans_preds.labels_ 

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(X)

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=20,
                         random_state=0,
                         batch_size=12)

In [ ]:
kmeans.fit(X)

In [ ]:
distortions = []
cluster_numbers = range(2, 20)
for k in cluster_numbers:
    k_means = KMeans(n_clusters=k, random_state=42)
    k_means.fit(X)
    distortions.append(sum(np.min(cdist(X, k_means.cluster_centers_, 'euclidean'), axis=1)) / len(X))

In [ ]:
X_line = [cluster_numbers[0], cluster_numbers[-1]]
Y_line = [distortions[0], distortions[-1]]

plt.figure(figsize=(10,10))
plt.plot(cluster_numbers, distortions, 'b-')
plt.plot(X_line, Y_line, 'r')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
res = pd.DataFrame()
res['Word'] = [val[1] for val in words_emb]
res['Emb'] = [val[0] for val in words_emb]
res['Concept'] = kmeans.labels_

In [ ]:
res.head()

In [ ]:
res.Concept.value_counts()

In [ ]:
res[res.Concept == 8].head(50)

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('../models/trained/word2vec.model')

In [ ]:
print(model.wv.most_similar('furto', topn=10))

In [ ]:
for concept in np.unique(kmeans_preds.labels_):
    print('Concept:', concept)
    print(model.wv.most_similar(positive=[np.mean(res[res.Concept==concept]['Emb'])], topn=10))

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('../models/trained/fasttext-sentiment.vec')

In [ ]:
print(model.most_similar('furto', topn=10))

In [ ]:
for concept in np.unique(kmeans_preds.labels_):
    print('Concept:', concept)
    print(model.most_similar(positive=[np.mean(res[res.Concept==concept]['Emb'])], topn=10))

In [ ]:
import fasttext
model = fasttext.load_model('../models/trained/fasttext-sentiment.bin')

In [ ]:
print(model.get_nearest_neighbors('desporto'))


In [ ]:
# get all words from model
words = model.get_words()

with open('../models/trained/fasttext-sentiment.vec','w') as file_out:
    
    # the first line must contain number of total words and vector dimension
    file_out.write(str(len(words)) + " " + str(model.get_dimension()) + "\n")

    # line by line, you append vectors to VEC file
    for w in words:
        v = model.get_word_vector(w)
        vstr = ""
        for vi in v:
            vstr += " " + str(vi)
        try:
            file_out.write(w + vstr+'\n')
        except:
            pass

In [ ]:
for concept in np.unique(kmeans_preds.labels_):
    print('Concept:', concept)
    print(model.most_similar(positive=[np.mean(res[res.Concept==concept]['Emb'])], topn=10))


## Sentiment Analysis

In [ ]:
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load("pt_core_news_lg")
nlp.add_pipe('spacytextblob')
text = 'Este restaurante é uma vergonha!'
doc = nlp(text)
doc._.blob.polarity                         # Polarity: -0.125
doc._.blob.subjectivity                        # Subjectivity: 0.9
doc._.blob.sentiment_assessments.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
doc._.blob.ngrams()                       

In [ ]:
doc._.blob.polarity 

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob

nlp = spacy.load("pt_core_news_lg")
nlp.add_pipe('spacytextblob')
text = 'mau'
doc = nlp(text)
doc._.blob.polarity                            # Polarity: -0.125
doc._.blob.subjectivity                        # Subjectivity: 0.9
doc._.blob.sentiment_assessments.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
#doc._.blob.ngrams()      

In [ ]:
doc._.blob.sentiment_assessments.assessments

## External data

In [ ]:
import ast
from sklearn.model_selection import StratifiedShuffleSplit


In [ ]:
df = pd.read_csv('../data/external_corpus/concatenated.csv')

In [ ]:
df.head()

In [ ]:
def sentiment(val):
    if val < 3:
        return 'Negative'
    elif val == 3:
        return 'Neutral'
    elif val > 3:
        return 'Positive'

df['sentiment'] = df['rating'].apply(lambda val : sentiment(val))       

In [ ]:
df['dataset'].value_counts()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df[df.sentiment == 'Neutral']['']

In [ ]:
df = df[df.dataset.isin(['olist', 'b2w'])]

In [ ]:
df.rating.unique()

In [ ]:
df.polarity.unique()

In [ ]:
df['polarity'].value_counts()

In [ ]:
reviews = [(ast.literal_eval(doc), r) for doc, r in zip(df['review_text_tokenized'].to_list(), df['sentiment'].tolist())]
print(len(reviews))
data = [(doc, r) for doc, r in reviews if len(doc) > 0]
print(len(data))


In [ ]:
X = np.array([' '.join(val[0]) for val in data])
Y = np.array([val[1] for val in data])

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(X, Y ):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
X_train

In [ ]:
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from scipy.sparse import vstack, hstack
from sklearn.metrics import classification_report

In [ ]:
count_vect = TfidfVectorizer()
X_train_ = count_vect.fit_transform(X_train)

In [ ]:
#nb=naive_bayes.BernoulliNB()
#nb.fit(X_train,y_train)
#prob=nb.feature_log_prob_ #index 0 is positive

#r=prob[0]-prob[1]

#print('Weighing features')
#X_train=[x.multiply(r).tocsr() for x in X_train_NB]
#X_train=vstack(X_train)

X_test_=count_vect.transform(X_test)
#X_test=[x.multiply(r).tocsr() for x in X_test_pre]
#X_test=vstack(X_test)


In [ ]:
print('Training classifier')

svc = linear_model.LogisticRegression()
svc.fit(X_train_,y_train)

print('Testing classifier')
print('Accuracy=', svc.score(X_test_, y_test)*100)
print(classification_report(y_test, svc.predict(X_test_)))

In [ ]:
svc = linear_model.SGDClassifier(loss='hinge', class_weight='balanced')
svc.fit(X_train_, y_train)

print('Testing classifier')
print('Accuracy=',svc.score(X_test_, y_test)*100)
print(classification_report(y_test, svc.predict(X_test_)))

In [ ]:
import fasttext

In [ ]:
df_train = pd.DataFrame() 
df_train['sentiment'] = y_train
df_train['review'] = X_train

df_train['sentiment'] = df_train['sentiment'].apply(lambda val: f'__label__{val}')

np.savetxt('olist-train.txt', df_train.values, fmt = "%s")

print(df_train.shape)

df_test = pd.DataFrame() 
df_test['sentiment'] = y_test
df_test['review'] = X_test

df_test['sentiment'] = df_test['sentiment'].apply(lambda val: f'__label__{val}')

np.savetxt('olist-test.txt', df_test.values, fmt = "%s")

print(df_test.shape)

In [ ]:
model = fasttext.train_supervised(input="olist-train.txt", lr=0.05, epoch=50, wordNgrams=3)

In [ ]:
model.test("olist-train.txt")

In [ ]:
model.test("olist-test.txt")

In [ ]:
df_test['Pred'] = df_test['review'].apply(lambda val: model.predict(val)[0][0])

In [ ]:
df_test.head()

In [ ]:
print(classification_report(df_test['sentiment'], df_test['Pred']))